In [2]:
import time
import torch,gc
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as Data
from torch.utils.data import Dataset
import random
import numpy as np
from torch import tensor
from math import e

Nr=4;Nt=4;K=4
channel = torch.from_numpy(np.load('noisechannel.npy',allow_pickle=True))
imagechannel = channel.view(18100,Nr**2,Nt**2)
#input normalization
#Rotate
r90channel = torch.zeros(size = (18100,Nr**2,Nt**2),dtype = torch.cfloat)
r180channel = torch.zeros(size = (18100,Nr**2,Nt**2),dtype = torch.cfloat)
r270channel = torch.zeros(size = (18100,Nr**2,Nt**2),dtype = torch.cfloat)
for counter in range(0,18099):
    rochannel = imagechannel[counter,:,:]
    r90 = np.rot90(rochannel, 1).copy()
    r180 = np.rot90(rochannel, 2).copy()
    r270 = np.rot90(rochannel, 3).copy()
    r90channel[counter,:,:] = torch.from_numpy(r90)
    r180channel[counter,:,:] = torch.from_numpy(r180)
    r270channel[counter,:,:] = torch.from_numpy(r270)

# flip
vflip = torch.zeros(size = (18100,Nr**2,Nt**2),dtype = torch.cfloat)
hflip = torch.zeros(size = (18100,Nr**2,Nt**2),dtype = torch.cfloat)
for counter in range(0,18099):
    flipchannel = imagechannel[counter,:,:]
    vflip[counter,:,:] = torch.flip(flipchannel,[0])
    hflip[counter,:,:] = torch.flip(flipchannel,[1])    
 
    
#Gaussian blur
noiseamp = 10**-7
gnoise = noiseamp*(torch.randn(18100,Nr**2,Nt**2)+torch.randn(18100,Nr**2,Nt**2)*1j)
gchannel = imagechannel + gnoise

# tranpose and conjugate
Tchannel = torch.randn(18100,Nr**2,Nt**2)+torch.randn(18100,Nr**2,Nt**2)*1j
Cchannel = torch.randn(18100,Nr**2,Nt**2)+torch.randn(18100,Nr**2,Nt**2)*1j
Hchannel = torch.randn(18100,Nr**2,Nt**2)+torch.randn(18100,Nr**2,Nt**2)*1j
for counter in range(0,18099):
    Tchannel[counter,:,:] = imagechannel[counter,:,:].T
    Cchannel[counter,:,:] = imagechannel[counter,:,:].conj()
    Hchannel[counter,:,:] = (imagechannel[counter,:,:].T).conj()
    
#Cut out
Ncutout = 2
cochannel = channel/abs(channel)
for counter in range(0,18099):
    cutoutindex = random.sample(range(0,15),Ncutout)
    cochannel[cutoutindex[0],counter]=0
    cochannel[cutoutindex[1],counter]=0
cutoutchannel = cochannel.view(18100,Nr**2,Nt**2)

# Normalization
data1 = Cchannel/abs(Cchannel)
data2 = Hchannel /abs(Hchannel)

#user grouping
datax = torch.zeros(size=(int(18100/K),2*K,Nr**2,Nt**2),dtype = torch.cfloat)
datay = torch.zeros(size=(int(18100/K),2*K,Nr**2,Nt**2),dtype = torch.cfloat)
uindex = (np.random.permutation(len(imagechannel))).reshape(-1,K)
for i in range(len(uindex)):
    for j in range(K):
        datax[i,j,:,:] = torch.real(data1[uindex[i,j],:,:])
        datax[i,j+K,:,:] = torch.imag(data1[uindex[i,j],:,:])
        datay[i,j,:,:] = torch.real(data2[uindex[i,j],:,:])
        datay[i,j+K,:,:] = torch.imag(data1[uindex[i,j],:,:])
        
minibatch = 128
trainset = Data.TensorDataset(datax,datay)
train_loader = Data.DataLoader(dataset = trainset, batch_size = minibatch, shuffle=True, num_workers = 2, drop_last = True)

del channel,imagechannel,gchannel,Tchannel,Hchannel,data1,data2


#N_training = int(0.7*len(imagechannel))
#N_testing = len(imagechannel)- N_training
#training_index = random.sample(range(0,len(imagechannel)),N_training)
#testing_index = random.sample(range(0,len(imagechannel)),N_testing)
#train_loader = torch.utils.data.DataLoader(traindata, batch_size=64, shuffle=True)
#test_loader = torch.utils.data.DataLoader(testdata, batch_size=64, shuffle=True)9.3

E:\anaconda\envs\deepmimo\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
###### define the device to use
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

learning_rate = 0.001
class network(nn.Module):
    def __init__(self):
        super(network, self).__init__()
        self.conv1 = nn.Conv2d(8, 8, kernel_size=2, stride=1)
        self.bn1 = nn.BatchNorm2d(8)
        self.conv2 = nn.Conv2d(8, 16, kernel_size=2, stride=1)
        self.bn2 = nn.BatchNorm2d(16)
        self.conv3 = nn.Conv2d(16, 32, kernel_size=2, stride=1)
        self.conv4 = nn.Conv2d(32, 32, kernel_size=2, stride=1)
        self.conv5 = nn.Conv2d(32, 64, kernel_size=2, stride=1)
        self.pool = nn.MaxPool2d(4,4)
        self.fc1 = nn.Linear(256,512)
        self.relu1 = nn.ReLU(inplace=False)
        self.fc2 = nn.Linear(512,512)
        self.relu2 = nn.ReLU(inplace=False)
        self.fc3 = nn.Linear(512,256)
        self.relu3 = nn.ReLU(inplace=False)
        self.fc4 = nn.Linear(256,64)

        #self.softmax = nn.Softmax(dim=0)
        
    def forward(self, x):
        x = self.bn1(self.conv1(x))
        x = self.bn2(self.conv2(x))
        x = (self.conv3(x))
        x = (self.conv4(x))
        x = (self.conv5(x))
        x = self.pool(x)
        x = torch.flatten(x, 1)
        x = self.relu1(self.fc1(x))
        x = self.relu2(self.fc2(x))
        x = self.relu3(self.fc3(x))
        x= self.fc4(x)
        #x = self.softmax(x)
        return x

model = network().to(device)
#criterion = nn.CrossEntropyLoss()
#optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate,weight_decay=1e-2)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, betas=(0.9, 0.999), eps=1e-8, weight_decay=0.01,amsgrad=False)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, verbose=False, threshold=0.001, threshold_mode='rel', cooldown=0, min_lr=0.000001, eps=1e-08)
#ExpLR = torch.optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.1, last_epoch=-1)

In [6]:
num_epochs = 4
temp = 0.5
for epoch in range(num_epochs):
    for step, (batchX,batchY) in enumerate(train_loader):
        # set zeros
        start = time.time()
        output = (torch.zeros(2*minibatch,8).to(device)).detach()
        upper = ((torch.zeros(minibatch)).to(device)).detach()
        lower = ((torch.zeros(minibatch*2,minibatch*2)).to(device)).detach()
        loss = (torch.zeros(1).to(device)).detach()
        
        # get batch
        batch = torch.cat((batchX,batchY),0).to(device)
        batch = (batch).to(torch.float32).requires_grad_()
        output = model(batch)
        for i in range(int(len(batch)/2)):
            upper[i] = ((e**(((output[i,:] @ output[i+int(len(batch)/2),:])/(torch.norm(output[i,:])* torch.norm(output[i+int(len(batch)/2),:])))/temp)))
        for j in range(len(batch)):
            for k in range(len(batch)):
                lower[j,k] = (e**(((output[j,:] @ output[k,:])/(torch.norm(output[j,:])* torch.norm(output[k,:])))/temp))
        loss = torch.sum(-torch.log((upper.repeat(2))/(torch.sum(lower,dim=1)-(upper.repeat(2)))),dim=0)
        
        gc.collect()
        torch.cuda.empty_cache()
        #loss.requires_grad_(True)
        optimizer.zero_grad()
        #requires_grad = True
        loss.backward()
        optimizer.step()
        scheduler.step(loss)
        end = time.time()
        
        print('Epoch: ', epoch, '| Step: ', step, '| Loss: ',float(loss/len(batch)), '|Time',end-start)
        #print(model.state_dict()['conv1.weight'])
        print('Learning rate:',optimizer.param_groups[0]['lr'])

print('Training Finished')


Epoch:  0 | Step:  0 | Loss:  5.537646293640137 |Time 73.66018748283386
Learning rate: 0.001
Epoch:  0 | Step:  1 | Loss:  5.523474216461182 |Time 73.42072224617004
Learning rate: 0.001
Epoch:  0 | Step:  2 | Loss:  5.41547966003418 |Time 73.25115275382996
Learning rate: 0.001
Epoch:  0 | Step:  3 | Loss:  5.357720375061035 |Time 73.60224175453186
Learning rate: 0.001
Epoch:  0 | Step:  4 | Loss:  5.0622711181640625 |Time 73.94033455848694
Learning rate: 0.001
Epoch:  0 | Step:  5 | Loss:  5.221648216247559 |Time 74.82106399536133
Learning rate: 0.001
Epoch:  0 | Step:  6 | Loss:  5.08548641204834 |Time 74.04702401161194
Learning rate: 0.001
Epoch:  0 | Step:  7 | Loss:  4.7340240478515625 |Time 74.26147890090942
Learning rate: 0.001
Epoch:  0 | Step:  8 | Loss:  4.621762275695801 |Time 74.46994996070862
Learning rate: 0.001
Epoch:  0 | Step:  9 | Loss:  4.511107444763184 |Time 74.47091889381409
Learning rate: 0.001
Epoch:  0 | Step:  10 | Loss:  4.467177867889404 |Time 74.327303171157

Epoch:  2 | Step:  17 | Loss:  3.7495813369750977 |Time 134.9952142238617
Learning rate: 0.000125
Epoch:  2 | Step:  18 | Loss:  3.759194850921631 |Time 134.94215607643127
Learning rate: 0.000125
Epoch:  2 | Step:  19 | Loss:  3.7342236042022705 |Time 133.2136447429657
Learning rate: 0.000125
Epoch:  2 | Step:  20 | Loss:  3.735128879547119 |Time 135.95192503929138
Learning rate: 0.000125
Epoch:  2 | Step:  21 | Loss:  3.7397053241729736 |Time 132.85398817062378
Learning rate: 0.000125
Epoch:  2 | Step:  22 | Loss:  3.750375270843506 |Time 124.83112621307373
Learning rate: 0.000125
Epoch:  2 | Step:  23 | Loss:  3.758948802947998 |Time 128.7159125804901
Learning rate: 0.000125
Epoch:  2 | Step:  24 | Loss:  3.7396416664123535 |Time 136.21109700202942
Learning rate: 0.000125
Epoch:  2 | Step:  25 | Loss:  3.7493011951446533 |Time 134.47489738464355
Learning rate: 6.25e-05
Epoch:  2 | Step:  26 | Loss:  3.738426446914673 |Time 136.95299243927002
Learning rate: 6.25e-05
Epoch:  2 | Step: 

KeyboardInterrupt: 

In [34]:
print(Tchannel.shape)






torch.Size([18100, 16, 16])


In [7]:
print([x.grad for x in optimizer.param_groups[0]['params']]) #check grad
for name in model.state_dict():
  print(name)
print(model.state_dict()['conv1.weight']) # check weights
print(realbatch[128,0,:,:].T)

[tensor([[[[nan, nan],
          [nan, nan]],

         [[nan, nan],
          [nan, nan]],

         [[nan, nan],
          [nan, nan]],

         [[nan, nan],
          [nan, nan]],

         [[nan, nan],
          [nan, nan]],

         [[nan, nan],
          [nan, nan]],

         [[nan, nan],
          [nan, nan]],

         [[nan, nan],
          [nan, nan]]],


        [[[nan, nan],
          [nan, nan]],

         [[nan, nan],
          [nan, nan]],

         [[nan, nan],
          [nan, nan]],

         [[nan, nan],
          [nan, nan]],

         [[nan, nan],
          [nan, nan]],

         [[nan, nan],
          [nan, nan]],

         [[nan, nan],
          [nan, nan]],

         [[nan, nan],
          [nan, nan]]],


        [[[nan, nan],
          [nan, nan]],

         [[nan, nan],
          [nan, nan]],

         [[nan, nan],
          [nan, nan]],

         [[nan, nan],
          [nan, nan]],

         [[nan, nan],
          [nan, nan]],

         [[nan, nan],
       

NameError: name 'realbatch' is not defined